# PART 3: MACHINE LEARNING
___

## Data and Library Imports

In [0]:
# Importing libraries
import numpy as np
import pandas as pd

from pyspark.sql.functions import col, date_format, concat_ws, udf, hour, avg, sqrt, lit, count, when, monotonically_increasing_id, row_number, min, max
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from pyspark.ml.evaluation import RegressionEvaluator

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

In [0]:
# Declaring Constants
catalog = "workspace"
schema  = "bde"
volume  = "assignment2"
BATCH_SIZE = 500000

___
# 1. Calculating the baseline RMSE for the model

In [0]:
# Importing the table from Part 2 - Question 3 
avg_df = spark.read.table(f"{catalog}.{schema}.Avg_Fares")
display(avg_df.limit(5))

Taxi_Colour,PICKUP_BOROUGH,DROPOFF_BOROUGH,MONTH,DAY_OF_WEEK,HOUR_OF_DAY,TOTAL_TRIPS,AVERAGE_DISTANCE,AVERAGE_AMOUNT_PER_TRIP,TOTAL_AMOUNT_PAID
Yellow,Manhattan,Manhattan,Jan,Friday,19,728332,1.8083952922568265,14.395360824463781,1.0484601940003354E7
Yellow,Manhattan,Manhattan,Jan,Friday,18,721301,1.7870492623745136,13.78119719784461,9940391.320002515
Yellow,Manhattan,Manhattan,Jan,Thursday,18,707053,1.764998833185064,14.015550036563027,9909736.700001998
Yellow,Manhattan,Manhattan,Jan,Wednesday,18,699603,1.7598216274086829,13.84365593057982,9685063.220001433
Yellow,Manhattan,Manhattan,Feb,Friday,19,687758,1.8273683185073744,14.192345388933493,9760899.080002122


In [0]:
# Rename the columns in the dataframe to match the taxi dataframe
avg_df = avg_df.withColumnRenamed(
            "PICKUP_BOROUGH", "PU_Borough"
        ).withColumnRenamed(
            "DROPOFF_BOROUGH", "DO_Borough"
        )

display(avg_df.limit(5))

Taxi_Colour,PU_Borough,DO_Borough,MONTH,DAY_OF_WEEK,HOUR_OF_DAY,TOTAL_TRIPS,AVERAGE_DISTANCE,AVERAGE_AMOUNT_PER_TRIP,TOTAL_AMOUNT_PAID
Yellow,Manhattan,Manhattan,Jan,Friday,19,728332,1.8083952922568265,14.395360824463781,1.0484601940003354E7
Yellow,Manhattan,Manhattan,Jan,Friday,18,721301,1.7870492623745136,13.78119719784461,9940391.320002515
Yellow,Manhattan,Manhattan,Jan,Thursday,18,707053,1.764998833185064,14.015550036563027,9909736.700001998
Yellow,Manhattan,Manhattan,Jan,Wednesday,18,699603,1.7598216274086829,13.84365593057982,9685063.220001433
Yellow,Manhattan,Manhattan,Feb,Friday,19,687758,1.8273683185073744,14.192345388933493,9760899.080002122


In [0]:
# Import the taxi dataframe

taxi_df = spark.read.table(f"{catalog}.{schema}.NYC_Taxi_Data")
display(taxi_df.limit(5))

VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,taxi_colour,trip_dur_hrs,trip_speed_kmph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone
1,2015-01-01T00:11:33.000Z,2015-01-01T00:16:48.000Z,N,1.0,41,166,1.0,1.0,5.7,0.5,0.5,1.4,0.0,null,0.0,8.4,1.0,null,null,null,Yellow,0.0875,18.392457142857143,Manhattan,Central Harlem,Boro Zone,Manhattan,Morningside Heights,Boro Zone
1,2015-01-01T00:18:24.000Z,2015-01-01T00:24:20.000Z,N,1.0,166,238,1.0,0.9,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.09888888888888889,14.646802247191012,Manhattan,Morningside Heights,Boro Zone,Manhattan,Upper West Side North,Yellow Zone
1,2015-01-01T00:26:19.000Z,2015-01-01T00:41:06.000Z,N,1.0,238,162,1.0,3.5,13.2,0.5,0.5,2.9,0.0,null,0.0,17.4,1.0,null,null,null,Yellow,0.24638888888888888,22.860974069898536,Manhattan,Upper West Side North,Yellow Zone,Manhattan,Midtown East,Yellow Zone
1,2015-01-01T00:45:26.000Z,2015-01-01T00:53:20.000Z,N,1.0,162,263,1.0,2.1,8.2,0.5,0.5,2.37,0.0,null,0.0,11.87,1.0,null,null,null,Yellow,0.13166666666666665,25.66795443037975,Manhattan,Midtown East,Yellow Zone,Manhattan,Yorkville West,Yellow Zone
1,2015-01-01T00:59:21.000Z,2015-01-01T01:05:24.000Z,N,1.0,236,141,1.0,1.0,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.10083333333333333,15.960396694214877,Manhattan,Upper East Side North,Yellow Zone,Manhattan,Lenox Hill West,Yellow Zone


In [0]:
# Create new columns within the taxi dataframe to match the grouped columns in the avg_df dataframe

taxi_df_join = taxi_df.withColumn(
        "MONTH", date_format("Pickup_Datetime", "MMM")
    ).withColumn(
        "DAY_OF_WEEK", date_format("Pickup_Datetime", "EEEE")
    ).withColumn(
        "HOUR_OF_DAY", hour("Pickup_Datetime")
    )

In [0]:
# Join the two dataframes
joined_df = taxi_df_join.join(
    avg_df,
    on=["Taxi_Colour", "PU_Borough", "DO_Borough", "MONTH", "DAY_OF_WEEK", "HOUR_OF_DAY"],
    how="left"
)

In [0]:
# Provide an alias name for the actual (Total_Amount) and prediction (Average_Amount_Per_Trip) columns

baseline_df = joined_df.select(
    col("Total_Amount").alias("y_true"),
    col("AVERAGE_AMOUNT_PER_TRIP").alias("y_pred")
)

In [0]:
# Create the test baseline dataset which is from Oct to Dec 2024

baseline_df_test = baseline_df.filter(
    col("pickup_datetime").between("2024-10-01", "2024-12-31")
)

In [0]:
# Filter the DataFrame by date range and remove rows with unrealistic 'total_amount' values

baseline_df_test_cleaned = baseline_df_test.filter(
    (col("total_amount") >= 1) & (col("total_amount") <= 1000)
)

In [0]:
# Calculating the RMSE

# Calculate the squared error of the baseline test dataframe
se_df = baseline_df_test_cleaned.withColumn(
    "se", (col("y_true") - col("y_pred"))**2
)

# Calculate the RMSE value
rmse_val = se_df.agg(sqrt(avg("se")).alias("rmse")).collect()[0]["rmse"]
print(f"Baseline RMSE: {rmse_val:.2f}")

Baseline RMSE: 16.78


___
# 2. Creating and Training Machine Learning Models
## a. Sampling the training and validation dataset using four features
- Time of day and Seasons - Capture temporal information
- Pickup and Dropoff Borough - Capture spatial information
- Trip Distance - Capture trip distance information 

In [0]:
# a. Capturing temporal information
# i. Time of Day

def time_of_day(hr):
    if 0 <= hr < 6:
        return "Early Morning"
    elif 6 <= hr < 10:
        return "Morning Rush"
    elif 10 <= hr < 16:
        return "Day"
    elif 16 <= hr < 20:
        return "Evening Rush"
    else:
        return "Night"


tod_udf = udf(time_of_day, StringType())

# ii. Seasons

def season(month):
    if month in ["December", "January", "February"]:
        return "Winter"
    elif month in ["March", "April", "May"]:
        return "Spring"
    elif month in ["June", "July", "August"]:
        return "Summer"
    elif month in ["September", "October", "November"]:
        return "Autumn"
    
season_udf = udf(season, StringType())

taxi_df = taxi_df.withColumn(
    'Time_Of_Day', tod_udf(hour("pickup_datetime"))
).withColumn(
    'Season', season_udf(date_format("pickup_datetime", "MMMM"))
)

display(taxi_df.limit(5))


VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,taxi_colour,trip_dur_hrs,trip_speed_kmph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,Time_Of_Day,Season
1,2015-01-01T00:11:33.000Z,2015-01-01T00:16:48.000Z,N,1.0,41,166,1.0,1.0,5.7,0.5,0.5,1.4,0.0,null,0.0,8.4,1.0,null,null,null,Yellow,0.0875,18.392457142857143,Manhattan,Central Harlem,Boro Zone,Manhattan,Morningside Heights,Boro Zone,Early Morning,Winter
1,2015-01-01T00:18:24.000Z,2015-01-01T00:24:20.000Z,N,1.0,166,238,1.0,0.9,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.09888888888888889,14.646802247191012,Manhattan,Morningside Heights,Boro Zone,Manhattan,Upper West Side North,Yellow Zone,Early Morning,Winter
1,2015-01-01T00:26:19.000Z,2015-01-01T00:41:06.000Z,N,1.0,238,162,1.0,3.5,13.2,0.5,0.5,2.9,0.0,null,0.0,17.4,1.0,null,null,null,Yellow,0.24638888888888888,22.860974069898536,Manhattan,Upper West Side North,Yellow Zone,Manhattan,Midtown East,Yellow Zone,Early Morning,Winter
1,2015-01-01T00:45:26.000Z,2015-01-01T00:53:20.000Z,N,1.0,162,263,1.0,2.1,8.2,0.5,0.5,2.37,0.0,null,0.0,11.87,1.0,null,null,null,Yellow,0.13166666666666665,25.66795443037975,Manhattan,Midtown East,Yellow Zone,Manhattan,Yorkville West,Yellow Zone,Early Morning,Winter
1,2015-01-01T00:59:21.000Z,2015-01-01T01:05:24.000Z,N,1.0,236,141,1.0,1.0,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.10083333333333333,15.960396694214877,Manhattan,Upper East Side North,Yellow Zone,Manhattan,Lenox Hill West,Yellow Zone,Early Morning,Winter


In [0]:
# b. Capturing spatial information

taxi_df = taxi_df.withColumn(
    "PU_DO_Borough", concat_ws("_", col("PU_Borough"), col("DO_Borough"))
)

display(taxi_df.limit(5))

VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,taxi_colour,trip_dur_hrs,trip_speed_kmph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,Time_Of_Day,Season,PU_DO_Borough
1,2015-01-01T00:11:33.000Z,2015-01-01T00:16:48.000Z,N,1.0,41,166,1.0,1.0,5.7,0.5,0.5,1.4,0.0,null,0.0,8.4,1.0,null,null,null,Yellow,0.0875,18.392457142857143,Manhattan,Central Harlem,Boro Zone,Manhattan,Morningside Heights,Boro Zone,Early Morning,Winter,Manhattan_Manhattan
1,2015-01-01T00:18:24.000Z,2015-01-01T00:24:20.000Z,N,1.0,166,238,1.0,0.9,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.09888888888888889,14.646802247191012,Manhattan,Morningside Heights,Boro Zone,Manhattan,Upper West Side North,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan
1,2015-01-01T00:26:19.000Z,2015-01-01T00:41:06.000Z,N,1.0,238,162,1.0,3.5,13.2,0.5,0.5,2.9,0.0,null,0.0,17.4,1.0,null,null,null,Yellow,0.24638888888888888,22.860974069898536,Manhattan,Upper West Side North,Yellow Zone,Manhattan,Midtown East,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan
1,2015-01-01T00:45:26.000Z,2015-01-01T00:53:20.000Z,N,1.0,162,263,1.0,2.1,8.2,0.5,0.5,2.37,0.0,null,0.0,11.87,1.0,null,null,null,Yellow,0.13166666666666665,25.66795443037975,Manhattan,Midtown East,Yellow Zone,Manhattan,Yorkville West,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan
1,2015-01-01T00:59:21.000Z,2015-01-01T01:05:24.000Z,N,1.0,236,141,1.0,1.0,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.10083333333333333,15.960396694214877,Manhattan,Upper East Side North,Yellow Zone,Manhattan,Lenox Hill West,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan


In [0]:
# c. Capturing trip distance information

def distance(trip_dist):
    if 0 <= trip_dist < 2:
        return "Short"
    elif 2 <= trip_dist < 10:
        return "Medium"
    else:
        return "Long"

dist_udf = udf(distance, StringType())

taxi_df = taxi_df.withColumn(
    "Distance_Bin", dist_udf(col("trip_distance"))
)

display(taxi_df.limit(5))

VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,taxi_colour,trip_dur_hrs,trip_speed_kmph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,Time_Of_Day,Season,PU_DO_Borough,Distance_Bin
1,2015-01-01T00:11:33.000Z,2015-01-01T00:16:48.000Z,N,1.0,41,166,1.0,1.0,5.7,0.5,0.5,1.4,0.0,null,0.0,8.4,1.0,null,null,null,Yellow,0.0875,18.392457142857143,Manhattan,Central Harlem,Boro Zone,Manhattan,Morningside Heights,Boro Zone,Early Morning,Winter,Manhattan_Manhattan,Short
1,2015-01-01T00:18:24.000Z,2015-01-01T00:24:20.000Z,N,1.0,166,238,1.0,0.9,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.09888888888888889,14.646802247191012,Manhattan,Morningside Heights,Boro Zone,Manhattan,Upper West Side North,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan,Short
1,2015-01-01T00:26:19.000Z,2015-01-01T00:41:06.000Z,N,1.0,238,162,1.0,3.5,13.2,0.5,0.5,2.9,0.0,null,0.0,17.4,1.0,null,null,null,Yellow,0.24638888888888888,22.860974069898536,Manhattan,Upper West Side North,Yellow Zone,Manhattan,Midtown East,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan,Medium
1,2015-01-01T00:45:26.000Z,2015-01-01T00:53:20.000Z,N,1.0,162,263,1.0,2.1,8.2,0.5,0.5,2.37,0.0,null,0.0,11.87,1.0,null,null,null,Yellow,0.13166666666666665,25.66795443037975,Manhattan,Midtown East,Yellow Zone,Manhattan,Yorkville West,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan,Medium
1,2015-01-01T00:59:21.000Z,2015-01-01T01:05:24.000Z,N,1.0,236,141,1.0,1.0,6.0,0.5,0.5,0.0,0.0,null,0.0,7.3,3.0,null,null,null,Yellow,0.10083333333333333,15.960396694214877,Manhattan,Upper East Side North,Yellow Zone,Manhattan,Lenox Hill West,Yellow Zone,Early Morning,Winter,Manhattan_Manhattan,Short


In [0]:
# Create the train test split
train_valid_df = taxi_df.filter(~col("pickup_datetime").between("2024-10-01", "2024-12-31"))
test_df = taxi_df.filter(col("pickup_datetime").between("2024-10-01", "2024-12-31"))

In [0]:
# Define the features which will be used to stratify the dataset
group_cols = ["Time_Of_Day", "Season", "PU_DO_Borough", "Distance_Bin"]

# Create a single key column with "-" as separator
train_valid_df = train_valid_df.withColumn(
    "group_key", concat_ws("-", *group_cols)
)

# Count number of rows per group
group_counts = train_valid_df.groupBy("group_key").count().orderBy("count", ascending=False)

# Show the results
group_counts.show(50, truncate=False)  # Show top 50 groups

+-----------------------------------------------+--------+
|group_key                                      |count   |
+-----------------------------------------------+--------+
|Day-Winter-Manhattan_Manhattan-Short           |43625435|
|Day-Spring-Manhattan_Manhattan-Short           |41931684|
|Day-Autumn-Manhattan_Manhattan-Short           |39610521|
|Day-Summer-Manhattan_Manhattan-Short           |38148980|
|Evening Rush-Winter-Manhattan_Manhattan-Short  |31952860|
|Evening Rush-Spring-Manhattan_Manhattan-Short  |30996227|
|Evening Rush-Autumn-Manhattan_Manhattan-Short  |29115283|
|Evening Rush-Summer-Manhattan_Manhattan-Short  |28067102|
|Night-Spring-Manhattan_Manhattan-Short         |24103388|
|Night-Winter-Manhattan_Manhattan-Short         |23787299|
|Night-Autumn-Manhattan_Manhattan-Short         |22345646|
|Night-Summer-Manhattan_Manhattan-Short         |21808504|
|Morning Rush-Spring-Manhattan_Manhattan-Short  |19877373|
|Morning Rush-Winter-Manhattan_Manhattan-Short  |1970894

In [0]:
# Create the stratified sample

# Compute global total and fractions for ~ 22500000 rows
total_rows = train_valid_df.count()
target_rows = 22500000
fraction = target_rows / total_rows

# Convert to dictionary for sampleBy
fractions_dict = {row["group_key"]: fraction for row in group_counts.collect()}

# Perform stratified sampling
sampled_df = train_valid_df.stat.sampleBy(col = "group_key", fractions = fractions_dict, seed = 42)

In [0]:
# Print the number of instances in the sampled dataset
sampled_df.count()

22500900

In [0]:
# Print the number of instances in the test dataset
test_df.count()

9670185

In [0]:
# Save the sampled dataset as a table
sampled_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.sampled_df")

In [0]:
# Save the test dataset as a table
test_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalog}.{schema}.test_df")

## b. Data Import

In [0]:
# Import the sampled dataset
df_spark = spark.table(f"{catalog}.{schema}.sampled_df")

In [0]:
# Print the shape of the dataset
df_shape = (df_spark.count(), len(df_spark.columns))
print("Shape of the dataframe: ", df_shape)

Shape of the dataframe:  (22498896, 35)


In [0]:
# Display the top 5 rows of the dataset
display(df_spark.limit(5))

VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,taxi_colour,trip_dur_hrs,trip_speed_kmph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,Time_Of_Day,Season,PU_DO_Borough,Distance_Bin,group_key
2,2020-03-02T12:02:43.000Z,2020-03-02T12:10:34.000Z,N,1.0,75,75,1.0,1.05,7.0,0.0,0.5,0.0,0.0,null,0.3,7.8,2.0,1.0,0.0,null,Green,0.13083333333333333,12.915722292993632,Manhattan,East Harlem South,Boro Zone,Manhattan,East Harlem South,Boro Zone,Day,Spring,Manhattan_Manhattan,Short,Day-Spring-Manhattan_Manhattan-Short
1,2020-03-02T12:53:34.000Z,2020-03-02T13:06:45.000Z,N,1.0,82,129,1.0,0.9,9.0,0.0,0.5,0.0,0.0,null,0.3,9.8,2.0,1.0,0.0,null,Green,0.21972222222222224,6.5919868520859675,Queens,Elmhurst,Boro Zone,Queens,Jackson Heights,Boro Zone,Day,Spring,Queens_Queens,Short,Day-Spring-Queens_Queens-Short
2,2020-03-02T12:01:18.000Z,2020-03-02T12:06:03.000Z,N,1.0,75,74,1.0,0.95,5.5,0.0,0.5,1.89,0.0,null,0.3,8.19,1.0,1.0,0.0,null,Green,0.07916666666666666,19.31208,Manhattan,East Harlem South,Boro Zone,Manhattan,East Harlem North,Boro Zone,Day,Spring,Manhattan_Manhattan,Short,Day-Spring-Manhattan_Manhattan-Short
2,2020-03-02T12:49:38.000Z,2020-03-02T12:52:05.000Z,N,1.0,242,183,1.0,0.59,4.0,0.0,0.5,0.0,0.0,null,0.3,4.8,1.0,1.0,0.0,null,Green,0.04083333333333333,23.253320816326532,Bronx,Van Nest/Morris Park,Boro Zone,Bronx,Pelham Bay,Boro Zone,Day,Spring,Bronx_Bronx,Short,Day-Spring-Bronx_Bronx-Short
1,2020-03-02T12:28:24.000Z,2020-03-02T12:40:54.000Z,N,1.0,74,236,1.0,2.5,11.0,2.75,0.5,2.0,0.0,null,0.3,16.55,1.0,1.0,2.75,null,Green,0.20833333333333334,19.312079999999998,Manhattan,East Harlem North,Boro Zone,Manhattan,Upper East Side North,Yellow Zone,Day,Spring,Manhattan_Manhattan,Medium,Day-Spring-Manhattan_Manhattan-Medium


## c. Data Cleaning
### i. Handling Null Values


In [0]:
# Checking for null values
null_counts = df_spark.select([
    count(when(col(c).isNull(), c)).alias(c)
    for c in df_spark.columns
])

display(null_counts)

VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,taxi_colour,trip_dur_hrs,trip_speed_kmph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,Time_Of_Day,Season,PU_DO_Borough,Distance_Bin,group_key
0,0,0,1173575,0,0,0,0,0,0,0,0,0,0,22498813,1583031,0,0,20670369,16956290,20991490,0,0,0,0,0,0,0,0,0,0,0,0,0,0


- #### Handling 'improvement_surcharge'

In [0]:
# Create a function to handle null values in the improvement_surcharge feature
def fill_improvement_surcharge(row):
    if pd.notnull(row['improvement_surcharge']):
        return row['improvement_surcharge']
    year = row['pickup_datetime'].year
    month = row['pickup_datetime'].month
    day = row['pickup_datetime'].day
    if year < 2015:
        return 0.0
    elif year <= 2022 and (month < 12 or (month == 12 and day <= 19)):
        return 0.30
    else:
        return 1.00

- #### Handling 'congestion_surcharge'

In [0]:
# Create a function to handle null values in the congestion_surcharge feature
def fill_congestion_surcharge(row):
    if pd.notnull(row['congestion_surcharge']):
        return row['congestion_surcharge']
    year = row['pickup_datetime'].year
    taxi_col = row['taxi_colour']
    if year < 2019:
        return 0.0
    else:
      if taxi_col == 'yellow':
        return 2.5
      else:
        return 2.75

- #### Handling 'ehail_fee' and 'airport_fee'

In [0]:
# Create a list to handle the remaining fee columns which will be filled with 0.0
handle_fee_cols = ['ehail_fee', 'airport_fee']

- #### Handling other features

In [0]:
# Drop the remaining features with null values as they don't contribute to the target variable
remaining_features_with_nulls = ['trip_type', 'store_and_fwd_flag']

### ii Handling Redundant Features

In [0]:
# Create a list with all the redundant features from the dataframe
redundant_features = [
    'pickup_datetime', 'dropoff_datetime', 'VendorID', 'RatecodeID', 'PULocationID', 'DOLocationID', 'PU_Zone', 'PU_service_zone', 
    'DO_Zone', 'DO_service_zone', 'Distance_Bin', 'PU_DO_Borough', 'group_key' 
]

### iii Handling Features with potential to introduce Data Leakage

In [0]:
# Create a list with all the features that could introduce data leakage
leaky_features = [
    'fare_amount', 'tolls_amount'
]

### iv Outlier Check

In [0]:
# Count the number (and percentage) of rows with target values outside the valid range [1, 1000], which could represent extreme outliers in `total_amount` that may distort model training

valid_range_count = df_spark.filter(
    (col("total_amount") < 1) | (col("total_amount") > 1000)
).count()

total_count = df_spark.count()

print(f"Rows not within the range of 1-1000: {valid_range_count}")
print(f"Total rows: {total_count}")
print(f"Percentage not within the range of 1-1000: {valid_range_count / total_count * 100}%")

Rows not within the range of 1-1000: 655
Total rows: 22498896
Percentage not within the range of 1-1000: 0.0029112539566385834%


### v Process the data in batches

In [0]:
# Create a list to store all the processed data batches
processed_batches = []

In [0]:
# Add unique row IDs
df_indexed = df_spark.withColumn("row_id", monotonically_increasing_id())

# Get min and max ID (instead of count, since IDs are sparse)
id_bounds = df_indexed.agg(min("row_id"), max("row_id")).collect()[0]
min_id, max_id = id_bounds[0], id_bounds[1]


In [0]:
# Define batch "step" size based on ID range
step = (max_id - min_id) // (df_indexed.count() // BATCH_SIZE + 1)

In [0]:
# Define batch number and the loop counter 'current'
batch_num = 0
current = min_id

In [0]:
# Create a loop which will process the data in batches

while current <= max_id:
    next_id = current + step

    # Extract rows for this batch
    batch_df_spark = (
        df_indexed
        .filter((col("row_id") >= current) & (col("row_id") < next_id))
        .drop("row_id")
    )

    # Ignore batches with no data
    if batch_df_spark.count() == 0:
        current = next_id
        continue

    # Convert from Spark to Pandas dataframe
    batch_df = batch_df_spark.toPandas()

    # a. Handling null values in 'improvement_surcharge'
    batch_df['improvement_surcharge'] = batch_df.apply(fill_improvement_surcharge, axis=1)

    # b. Handling null values in 'congestion_surcharge'
    batch_df['congestion_surcharge'] = batch_df.apply(fill_congestion_surcharge, axis=1)
    
    # c. Handling null values in ehail_fee and airport_type
    batch_df['ehail_fee'] = batch_df['ehail_fee'].fillna(0.0)
    batch_df['airport_fee'] = batch_df['airport_fee'].fillna(0.0)

    # d. Drop the remaining features with null values as they don't contribute to the target variable
    batch_df = batch_df.drop(remaining_features_with_nulls, axis=1)

    # d. Drop the redundant features from the dataframe
    batch_df = batch_df.drop(redundant_features, axis=1)

    # e. Drop the leaky features from the dataframe
    batch_df = batch_df.drop(leaky_features, axis=1)

    # f. Drop instances where the total amount has unrealistic values i.e. < 1 or > 1000
    batch_df = batch_df[(batch_df['total_amount'] >= 1) & (batch_df['total_amount'] <= 1000)]


    # g. Drop the duplicate features
    num_duplicates = batch_df.duplicated().sum()
    print(f"Number of duplicated rows in batch {batch_num}: {num_duplicates}")
    if num_duplicates > 0:
        batch_df = batch_df.drop_duplicates()
    

    # Append the processed data to the processed_batches list
    processed_batches.append(batch_df)

    batch_num += 1
    current = next_id


Number of duplicated rows in batch 0: 95679
Number of duplicated rows in batch 1: 95740
Number of duplicated rows in batch 2: 29117
Number of duplicated rows in batch 3: 18108
Number of duplicated rows in batch 4: 75890
Number of duplicated rows in batch 5: 73019
Number of duplicated rows in batch 6: 20599
Number of duplicated rows in batch 7: 175306
Number of duplicated rows in batch 8: 36337
Number of duplicated rows in batch 9: 46054
Number of duplicated rows in batch 10: 94439
Number of duplicated rows in batch 11: 319755
Number of duplicated rows in batch 12: 0


In [0]:
# Create the final dataframe by from the processed_batches list
final_df = pd.concat(processed_batches, ignore_index=True)

In [0]:
# Display the shape of the final dataframe
final_df.shape

(21418198, 18)

In [0]:
# Save the preporcessed pandas dataframe
final_df.to_csv(f"/Volumes/{catalog}/{schema}/{volume}/final_df.csv", index = False)

## d. Feature Splitting

In [0]:
# Reading the preprocessed pandas dataframe
final_df = pd.read_csv(f"/Volumes/{catalog}/{schema}/{volume}/final_df.csv")

In [0]:
# Splitting the data into features 'X' and target 'y'
X = final_df.drop('total_amount', axis=1)
y = final_df['total_amount']

## e. Train - Validation Split

In [0]:
# Train Validation Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## f. Categorical Encoding - Frequency Mapping

In [0]:
# Create a list with all the categorical features
categorical_features = X_train.select_dtypes(exclude=['int64', 'float64']).columns.tolist()
print(categorical_features)

['taxi_colour', 'PU_Borough', 'DO_Borough', 'Time_Of_Day', 'Season']


In [0]:
# Create a dictionary to store the frequency maps
freq_maps = {}

In [0]:
# Loop through each categorical feature and create a frequency map
for feat in categorical_features:
    freq_map = X_train[feat].value_counts().to_dict()
    freq_maps[feat] = freq_map

    X_train[feat] = X_train[feat].map(freq_map).fillna(0)
    X_val[feat] = X_val[feat].map(freq_map).fillna(0)

In [0]:
# Display the shape of the training set
X_train.shape

(17134558, 17)

In [0]:

# Display the shape of the validation set
X_val.shape

(4283640, 17)

## g. ML Modelling
### i. Linear Regression

In [0]:
# Instantiate and train the Linear Regression Model
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [0]:
# Make predictions on the validation set
y_pred_lr = lr.predict(X_val)

# Calculate the RMSE
rmse_lr = np.sqrt(mean_squared_error(y_val, y_pred_lr))
print(f"Linear Regression RMSE: {rmse_lr:.2f}")

Linear Regression RMSE: 3.49


### ii Decision Tree

In [0]:
# Instantiate and train the Decision Tree Model
dt = DecisionTreeRegressor()
dt.fit(X_train, y_train)

DecisionTreeRegressor()

In [0]:
# Make predictions on the validation set
y_pred_dt = dt.predict(X_val)

# Calculate the RMSE
rmse_dt = np.sqrt(mean_squared_error(y_val, y_pred_dt))
print(f"Decision Tree RMSE: {rmse_dt:.2f}")

Decision Tree RMSE: 2.91


___
# 3 Model Testing

In [0]:
# Import the test data and display the top 5 rows
test_df_spark = spark.table(f"{catalog}.{schema}.test_df")
display(test_df_spark.limit(5))

VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,airport_fee,taxi_colour,trip_dur_hrs,trip_speed_kmph,PU_Borough,PU_Zone,PU_service_zone,DO_Borough,DO_Zone,DO_service_zone,Time_Of_Day,Season,PU_DO_Borough,Distance_Bin
2,2024-12-18T11:50:48.000Z,2024-12-18T12:13:17.000Z,N,1.0,162,143,1.0,2.18,20.5,0.0,0.5,6.12,0.0,null,1.0,30.62,1.0,null,2.5,null,Yellow,0.37472222222222223,9.36256510007413,Manhattan,Midtown East,Yellow Zone,Manhattan,Lincoln Square West,Yellow Zone,Day,Winter,Manhattan_Manhattan,Medium
1,2024-12-18T11:00:30.000Z,2024-12-18T11:10:14.000Z,N,1.0,79,234,1.0,1.3,10.7,2.5,0.5,2.9,0.0,null,1.0,17.6,1.0,null,2.5,null,Yellow,0.1622222222222222,12.896765753424658,Manhattan,East Village,Yellow Zone,Manhattan,Union Sq,Yellow Zone,Day,Winter,Manhattan_Manhattan,Short
1,2024-12-18T11:47:48.000Z,2024-12-18T12:16:45.000Z,N,1.0,79,138,1.0,12.3,49.2,7.5,0.5,16.25,6.94,null,1.0,81.39,1.0,null,2.5,null,Yellow,0.4825,41.02566217616581,Manhattan,East Village,Yellow Zone,Queens,LaGuardia Airport,Airports,Day,Winter,Manhattan_Queens,Long
2,2024-12-18T11:12:19.000Z,2024-12-18T11:17:22.000Z,N,1.0,143,142,1.0,0.88,6.5,0.0,0.5,2.1,0.0,null,1.0,12.6,1.0,null,2.5,null,Yellow,0.08416666666666667,16.826366732673268,Manhattan,Lincoln Square West,Yellow Zone,Manhattan,Lincoln Square East,Yellow Zone,Day,Winter,Manhattan_Manhattan,Short
2,2024-12-18T11:50:26.000Z,2024-12-18T11:57:12.000Z,N,1.0,229,162,1.0,0.46,7.9,0.0,0.5,2.38,0.0,null,1.0,14.28,1.0,null,2.5,null,Yellow,0.11277777777777778,6.564204532019705,Manhattan,Sutton Place/Turtle Bay North,Yellow Zone,Manhattan,Midtown East,Yellow Zone,Day,Winter,Manhattan_Manhattan,Short


In [0]:
# Count the number (and percentage) of rows with target values outside the valid range [1, 1000], which could represent extreme outliers in `total_amount` that may distort model evaluation
valid_range_count = test_df_spark.filter(
    (col("total_amount") < 1) | (col("total_amount") > 1000)
).count()

total_count = test_df_spark.count()

print(f"Rows not within the range of 1-1000: {valid_range_count}")
print(f"Total rows: {total_count}")
print(f"Percentage not within the range of within 1-1000: {valid_range_count / total_count * 100}%")

Rows not within the range of 1-1000: 5
Total rows: 9670185
Percentage not within the range of within 1-1000: 5.1705318977868574e-05%


In [0]:
# Add unique row IDs
test_df_indexed = test_df_spark.withColumn("row_id", monotonically_increasing_id())

# Get min and max ID (instead of count, since IDs are sparse)
id_bounds = test_df_indexed.agg(min("row_id"), max("row_id")).collect()[0]
min_id, max_id = id_bounds[0], id_bounds[1]

In [0]:
# Define batch "step" size based on ID range
step = (max_id - min_id) // (test_df_indexed.count() // BATCH_SIZE + 1)

In [0]:
# Declare batch number and the loop counter 'current'
batch_num = 0
current = min_id

In [0]:
# Declare lists to concatenate the results towards the end
all_preds_lr = []
all_preds_dt = []

all_true = []


In [0]:
# Remove the grouping key from the redundant features as it is not present in the test dataset
redundant_features.remove('group_key')

In [0]:
# Create a loop which will process the data in batches

while current <= max_id:
    next_id = current + step

    # Extract rows for this batch
    batch_df_spark = (
        test_df_indexed
        .filter((col("row_id") >= current) & (col("row_id") < next_id))
        .drop("row_id")
    )

    # Ignore batches with no data
    if batch_df_spark.count() == 0:
        current = next_id
        continue
    
    # Convert from spark to pandas dataframe
    batch_df = batch_df_spark.toPandas()

    # a. Handling null values in 'improvement_surcharge'
    batch_df['improvement_surcharge'] = batch_df.apply(fill_improvement_surcharge, axis=1)

    # b. Handling null values in 'congestion_surcharge'
    batch_df['congestion_surcharge'] = batch_df.apply(fill_congestion_surcharge, axis=1)

    # c. Handling null values in ehail_fee and airport_type
    batch_df['ehail_fee'] = batch_df['ehail_fee'].fillna(0.0)
    batch_df['airport_fee'] = batch_df['airport_fee'].fillna(0.0)

    # d. Drop the remaining features with null values as they don't contribute to the target variable
    batch_df = batch_df.drop(remaining_features_with_nulls, axis=1)

    # e. Drop the redundant features from the dataframe
    batch_df = batch_df.drop(redundant_features, axis=1)

    # f. Drop the leaky features from the dataframe
    batch_df = batch_df.drop(leaky_features, axis=1)

    # g. Drop instances where the total amount has unrealistic values i.e. < 1 or > 1000
    batch_df = batch_df[(batch_df['total_amount'] >= 1) & (batch_df['total_amount'] <= 1000)]

    # h. Drop the duplicate features
    num_duplicates = batch_df.duplicated().sum()
    print(f"Number of duplicated rows in batch {batch_num}: {num_duplicates}")
    if num_duplicates > 0:
        batch_df = batch_df.drop_duplicates()

    # i. Separate target
    X_batch = batch_df.drop('total_amount', axis=1)
    y_batch = batch_df['total_amount']

    # j. Frequency encoding with unseen categories handled as 0
    for feat, freq_map in freq_maps.items():
        X_batch[feat] = X_batch[feat].map(lambda x: freq_map.get(x, 0))

    # k. Align columns to training
    X_batch = X_batch[X_train.columns]

    # l. Predict
    y_pred_lr = lr.predict(X_batch)
    y_pred_dt = dt.predict(X_batch)

    all_preds_lr.extend(y_pred_lr)
    all_preds_dt.extend(y_pred_dt)
    all_true.extend(y_batch)

    batch_num += 1
    current = next_id


Number of duplicated rows in batch 0: 147526
Number of duplicated rows in batch 1: 71
Number of duplicated rows in batch 2: 246464
Number of duplicated rows in batch 3: 291
Number of duplicated rows in batch 4: 62381
Number of duplicated rows in batch 5: 501
Number of duplicated rows in batch 6: 25440
Number of duplicated rows in batch 7: 196
Number of duplicated rows in batch 8: 0


In [0]:
# For Linear Regression
test_rmse_lr = np.sqrt(mean_squared_error(all_true, all_preds_lr))
print(f"Linear Regression Test RMSE: {test_rmse_lr:.2f}")

Linear Regression Test RMSE: 7.07


In [0]:
# For Decision Tree
test_rmse_dt = np.sqrt(mean_squared_error(all_true, all_preds_dt))
print(f"Decision Tree RMSE: {test_rmse_dt:.2f}")

Decision Tree RMSE: 4.55
